In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset.columns

Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')

In [5]:
dataset=pd.get_dummies(dataset, drop_first=True)

In [6]:
dataset


,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [8]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [9]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [10]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [11]:
indep.shape

(400, 3)

In [12]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
from sklearn.model_selection import GridSearchCV
#LogisticRegresson parameter
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 


#GridSearchCV parameter
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [17]:
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)



In [18]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for the best parameter {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 100}: 0.9031880225910075


In [19]:
print("The confusion matricx:\n",cm)

The confusion matricx:
 [[78  7]
 [ 6 43]]


In [20]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9661464585834334

In [22]:
table=pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025625,0.007139,0.001190,0.001739,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.847141,0.868752,0.851527,0.943699,0.942166,0.890657,0.043293,8
1,0.207305,0.011024,0.021442,0.008736,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.867478,0.907177,0.851527,0.925272,0.962264,0.902744,0.039841,1
2,0.021130,0.004269,0.001027,0.002055,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.885356,0.868752,0.851527,0.925272,0.961755,0.898532,0.039955,3
3,0.191957,0.012994,0.024883,0.009066,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.888107,0.851527,0.925272,0.943699,0.895217,0.034591,4
4,0.021011,0.004519,0.002966,0.004382,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.850809,0.813179,0.886792,0.865054,0.856663,0.024578,12
5,0.262765,0.074103,0.031290,0.008538,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.907177,0.851527,0.925272,0.943699,0.894963,0.039030,5
6,0.052440,0.020933,0.014464,0.003534,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.826263,0.870047,0.833323,0.906166,0.924528,0.872065,0.038777,11
7,0.229335,0.018750,0.023303,0.008514,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.867478,0.888107,0.833323,0.925272,0.943699,0.891576,0.039594,6
8,0.022077,0.004418,0.007531,0.005417,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.868752,0.832483,0.906166,0.943699,0.875473,0.044524,10
9,0.230923,0.020488,0.028519,0.006617,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.888107,0.851527,0.925272,0.943699,0.891149,0.038579,7


# age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
childern_input=(input("childern:"))
sex_male_input=(input("sex_male 0 or 1"))
smoker_yes_input=(input("smoker yes 0 or 1"))
